In [1]:
"https://edition.cnn.com/markets/fear-and-greed"
"https://www.currentmarketvaluation.com/"
"https://www.oceans14.com.br/acoes/historico-pl-bovespa"

#fear and greed index
import requests
import json

headers = {
    'accept': '*/*',
    'accept-language': 'pt-BR,pt;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
    'if-none-match': 'W/2666142332818888819',
    'origin': 'https://edition.cnn.com',
    'priority': 'u=1, i',
    'referer': 'https://edition.cnn.com/',
    'sec-ch-ua': '"Microsoft Edge";v="141", "Not?A_Brand";v="8", "Chromium";v="141"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"Windows"',
    'sec-fetch-dest': 'empty',
    'sec-fetch-mode': 'cors',
    'sec-fetch-site': 'cross-site',
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36 Edg/141.0.0.0',
}

response = requests.get('https://production.dataviz.cnn.io/index/fearandgreed/graphdata', headers=headers)

In [2]:
overview = response.json()['fear_and_greed']
print(json.dumps(overview, indent=3))

{
   "score": 46.8857142857143,
   "rating": "neutral",
   "timestamp": "2025-10-10T15:07:04+00:00",
   "previous_close": 49.0571428571429,
   "previous_1_week": 53.6857142857143,
   "previous_1_month": 49.8571428571429,
   "previous_1_year": 70.28571428571429
}


In [3]:
import pandas as pd

df = pd.DataFrame(overview, index=[0])
df

,score,rating,timestamp,previous_close,previous_1_week,previous_1_month,previous_1_year
0,46.885714,neutral,2025-10-10T15:07:04+00:00,49.057143,53.685714,49.857143,70.285714


In [4]:
import pandas as pd
import scipy.stats as stats

# Função para calcular rating com média e desvio padrão
def get_rating_with_std(value: float, mean: float, std: float) -> str:
    if std == 0:
        return 'Neutral'  # Evitar divisão por zero
    z = (value - mean) / std
    percentile = stats.norm.cdf(z) * 100
    if percentile <= 25:
        return 'Extreme Fear'
    elif percentile <= 40:
        return 'Fear'
    elif percentile <= 60:
        return 'Neutral'
    elif percentile <= 75:
        return 'Greed'
    else:
        return 'Extreme Greed'


dfs = []

titles = {
    'fear_and_greed_historical': 'Fear & Greed Index Over Time',
    'market_momentum_sp500': 'S&P 500 Market Momentum',
    'market_momentum_sp125': 'S&P 125 Market Momentum',
    'stock_price_strength': 'Stock Price Strength',
    'stock_price_breadth': 'Stock Price Breadth',
    'put_call_options': 'Put/Call Options Ratio',
    'market_volatility_vix': 'Market Volatility (VIX)',
    'market_volatility_vix_50': 'VIX 50-day Moving Average',
    'junk_bond_demand': 'Junk Bond Demand',
    'safe_haven_demand': 'Safe Haven Demand'
}

for key in titles:
    data = response.json()[key]['data']
    df_aux = pd.DataFrame(data)[['x', 'y']]
    df_aux.rename(columns={'y': key}, inplace=True)
    df_aux['date'] = pd.to_datetime(df_aux['x'], unit='ms')
    df_aux.drop(columns=['x'], inplace=True)
    df_aux.set_index('date', inplace=True)
    
    mean = df_aux[key].mean()
    std = df_aux[key].std()
    
    df_aux[f'{key}_rating'] = df_aux[key].apply(lambda v: get_rating_with_std(v, mean, std))
    
    dfs.append(df_aux)

df = pd.concat(dfs, axis=1)
df.sort_index(inplace=True)


In [5]:
import plotly.graph_objects as go

def plot_data(df, key: str, title: str):
    """
    Plota uma métrica do DataFrame consolidado com cores por rating.
    
    df: DataFrame consolidado com colunas <key> e <key>_rating
    key: métrica que será plotada
    title: título do gráfico
    """
    # Preparar dados
    df_plot = df[[key, f"{key}_rating"]].copy()
    df_plot['color'] = df_plot[f"{key}_rating"].map({
        'Extreme Fear': 'darkred',
        'Fear': 'red',
        'Neutral': 'gold',
        'Greed': 'green',
        'Extreme Greed': 'darkgreen'
    })

    # Criar figura
    fig = go.Figure()

    # Linha contínua conectando os pontos
    fig.add_trace(go.Scatter(
        x=df_plot.index,
        y=df_plot[key],
        mode='lines+markers',
        line=dict(color='lightgray', width=2),
        marker=dict(color=df_plot['color'], size=8),
        text=[f"{y:.2f} ({r})" for y, r in zip(df_plot[key], df_plot[f"{key}_rating"])],
        hoverinfo='text+x'
    ))

    # Layout
    fig.update_layout(
        title=title,
        xaxis_title='Date',
        yaxis_title='Score',
        yaxis=dict(range=[df_plot[key].min() * 0.95, df_plot[key].max() * 1.05]),
        template='plotly_white'
    )

    fig.show()


In [ ]:
for key, title in titles.items():
    plot_data(df, key, title)

In [7]:


# Loop para plotar todos
for key, title in titles.items():
    plot_data(response, key=key, title=title)


TypeError: 'Response' object is not subscriptable